In [3]:
import numpy as np
import copy
import random 
from pathlib import Path

In [34]:
class RecurSimule:
    def __init__(self, cap, values, weights): # Initialisation des données
        self.capacity = cap
        self.values = values
        self.weights = weights
        self.objects = [((v, w), i) for i, (v, w) in enumerate(zip(values, weights))]
        self.best_solution=0
        self.best_value=0
       

    def initial_solution(self):
        solution = []
        items = self.objects.copy()
        c = self.capacity
        while c > 0 and len(items) > 1:
            myobject = items.pop(np.random.randint(0, len(items)-1))
            if myobject[0][1] <= c:
                solution.append(myobject[1])
            c -= myobject[0][1]
        return solution

    def voisinage(self,solution):

        objects_no_solution = [i for i in range(len(self.objects)) if i not in solution]
        o = objects_no_solution.pop(random.randint(0, len(objects_no_solution)-1))
        solution.append(o)
        if len(solution) == 1:
            return solution
        while self.evalution_weight(solution) > self.capacity and len(solution) > 1:
            solution.pop(random.randint(0, len(solution)-2))
        return solution
    def Metro_criterion(self, delta, t):
          
          return random.random() < np.exp(delta/t)

    def evaluation_values(self, solution):
            val = 0
            for obj in solution:
                val += self.objects[obj][0][0]
            return val

    def evalution_weight(self, solution):

        weight = 0
        for obj in solution:
            weight += self.objects[obj][0][1]
        return weight
    
    def temperature(self,t,k,step=10):
        return t - (k%step == 0) * step

    def solve(self, T0=300):
        S0 = self.initial_solution()
        t = T0
        i = 0
        last_best = 0
        max_iter = 1000
        self.best_solution  = S0
        self.best_value = self.evaluation_values(self.best_solution)
        while i < max_iter and t > 0:
            S1 = self.voisinage(S0)
            delta = self.evaluation_values(S1) - self.evaluation_values(S0)
            if delta > 0:
                S0 = S1
            elif self.Metro_criterion(delta, t):
                S0 = S1
            if self.evaluation_values(S0) > self.best_value:
                self.best_solution = S0.copy()
                self.best_value = self.evaluation_values(self.best_solution)
            
            t = self.temperature(t, i)
            i += 1
        return sum(self.values[self.best_solution])
    
capacite = 10
values = np.array([40, 50, 100, 95, 30])
weights = np.array([2, 3, 1, 5, 3])
kpp = RecurSimule(capacite, values, weights)
kpp.solve()


245

In [35]:
kpp.best_solution

[2, 3, 1]

In [36]:
kpp.best_value

245

In [47]:
def execute_all(directory):
    p=Path('./instances_01_KP/'+directory)
    print('Fichier           Optimum')
    nb=0
    for f in p.iterdir():
        first_row=np.loadtxt(f.absolute(),max_rows=1)
        fich=np.loadtxt(f.absolute(),max_rows=int(first_row[0])+1)
        clf=RecurSimule(fich[0][1],fich[1:,0],fich[1:,1])
        val=clf.solve()
        print(f'{f.name} -> {val}')

In [69]:
execute_all('low-dimensional')

Fichier           Optimum
f2_l-d_kp_20_878 -> 1018.0
f8_l-d_kp_23_10000 -> 9742.0
f4_l-d_kp_4_11 -> 23.0
f10_l-d_kp_20_879 -> 1014.0
f5_l-d_kp_15_375 -> 448.20093299999996
f3_l-d_kp_4_20 -> 35.0
f1_l-d_kp_10_269 -> 294.0
f6_l-d_kp_10_60 -> 52.0
f7_l-d_kp_7_50 -> 107.0
f9_l-d_kp_5_80 -> 69.0


In [39]:
execute_all('large_scale')

Fichier           Optimum
knapPI_3_100_1000_1 -> 1342.0
knapPI_3_2000_1000_1 -> 12173.0
knapPI_3_500_1000_1 -> 3247.0
knapPI_3_200_1000_1 -> 1443.0
knapPI_2_5000_1000_1 -> 26479.0
knapPI_1_500_1000_1 -> 4968.0
knapPI_3_10000_1000_1 -> 60700.0
knapPI_2_100_1000_1 -> 1145.0
knapPI_2_500_1000_1 -> 2739.0
knapPI_1_100_1000_1 -> 3422.0
knapPI_3_1000_1000_1 -> 6172.0
knapPI_1_1000_1000_1 -> 8511.0
knapPI_2_10000_1000_1 -> 50520.0
knapPI_2_2000_1000_1 -> 10798.0
knapPI_2_1000_1000_1 -> 5431.0
knapPI_1_10000_1000_1 -> 57671.0
knapPI_1_2000_1000_1 -> 14585.0
knapPI_2_200_1000_1 -> 1217.0
knapPI_1_5000_1000_1 -> 32632.0
knapPI_3_5000_1000_1 -> 30677.0
knapPI_1_200_1000_1 -> 3255.0


In [32]:
np.random.randint(0, 1)

0